In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.initializers import GlorotUniform, Zeros
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    Rescaling,
)
import numpy as np
import random
import os
import cv2
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import classification_report
import os
from tqdm import trange, tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
! unzip /content/drive/MyDrive/Dataset.zip

Output streaming troncato alle ultime 5000 righe.
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem21.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem210.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem211.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem212.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem213.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem214.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem215.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem216.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem217.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem218.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem219.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem22.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mildDem220.jpg  
  inflating: Alzheimer_s Dataset/train/MildDemented/mil

# Importing Dataset

## Split data for 3 Devices: alpha, beta, gamma

### Costants

In [7]:
LABELS = ["NonDemented", "VeryMildDemented", "MildDemented", "ModerateDemented"]
HOSPITAL_SPLIT = {"Alpha": 0.5, "Beta": 0.3, "Gamma": 0.2}
TRAIN_TEST_SPLIT = 0.30
RANDOM_SEED=42
NUM_EPOCHS=3
NUM_ROUNDS=5
BATCH_SIZE=64
SIMILARITY = 'single'
WIDTH = 176
HEIGHT = 208
DEPTH = 1

### Useful function

In [9]:
def set_reproducibility(seed=RANDOM_SEED):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
    tf.keras.utils.set_random_seed(seed)

In [10]:
def create_dataset(img_folder):
    img_data_array = []
    class_name = []

    for dir1 in os.listdir(img_folder):
      for idx, file in enumerate(os.listdir(os.path.join(img_folder, dir1))):
          image_path = os.path.join(img_folder, dir1, file)
          image = cv2.imread(image_path, 0)
          image = np.array(image)
          image = image.astype("float32")
          img_data_array.append(image)
          class_name.append(dir1)

    return img_data_array, class_name

In [12]:
DATASET_TRAIN_PATH = "/content/Alzheimer_s Dataset/train"
DATASET_TEST_PATH = "/content/Alzheimer_s Dataset/test"
VAL_SPLIT = 0.3

In [13]:
DATASET_PATH = "/content/Alzheimer_s Dataset/train"
def createHospitals():
    hospitals = {}

    # extract the image array and class name
    img_data, class_name = create_dataset(DATASET_TRAIN_PATH)
    img_data_test, class_name_test = create_dataset(DATASET_TEST_PATH)

    """
    target_dict = {
        "NonDemented": 0,
        "VeryMildDemented": 1,
        "MildDemented": 2,
        "ModerateDemented": 3,
    }
    """
    target_dict = {label: index for index, label in enumerate(LABELS)}

    target_val = [target_dict[class_name[i]] for i in range(len(class_name))]

    target_val_test = [target_dict[class_name_test[i]] for i in range(len(class_name_test))]

    X = np.array(img_data, np.float32)
    y = np.array(list(map(int, target_val)), np.float32)

    X_test = np.array(img_data_test, np.float32)
    y_test = np.array(list(map(int, target_val_test)), np.float32)

    rows = len(X)
    values_list = []
    for hospital_name in HOSPITAL_SPLIT:
        values_list += [hospital_name] * int(rows * HOSPITAL_SPLIT[hospital_name])

    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]

    df = pd.DataFrame({"X": list(X), "y": list(y)})
    if df.shape[0] != len(values_list):
        values_list.append("Gamma")
    df["hospital"] = values_list
    # df['hospital'] = df['hospital'].map(hospitals)

    dataset = dict.fromkeys(list(HOSPITAL_SPLIT.keys()))

    for hospital_name in HOSPITAL_SPLIT:
        X_h = df[df["hospital"] == hospital_name]["X"].to_numpy()
        y_h = df[df["hospital"] == hospital_name]["y"].to_numpy()

        X_h = np.stack(X_h, axis=0)
        y_h = np.stack(y_h, axis=0)

        dataset[hospital_name] = {}

        (
            X_test,
            X_val,
            y_test,
            y_val,
        ) = train_test_split(X_test, y_test, test_size=VAL_SPLIT, random_state=RANDOM_SEED)

        dataset[hospital_name]["X_train"] = np.expand_dims(np.array(X_h, np.float32), axis=-1)
        dataset[hospital_name]["y_train"] = tf.one_hot(y_h, depth=len(LABELS))
        dataset[hospital_name]["X_test"] = np.expand_dims(np.array(X_test, np.float32), axis=-1)
        dataset[hospital_name]["y_test"] = tf.one_hot(y_test, depth=len(LABELS))
        dataset[hospital_name]["X_val"] = np.expand_dims(np.array(X_val, np.float32), axis=-1)
        dataset[hospital_name]["y_val"] = tf.one_hot(y_val, depth=len(LABELS))

        hospitals[hospital_name] = dataset[hospital_name]

    """
    hospital_Alpha = Hospital("Alpha", dataset_Alpha)
    hospital_Beta = Hospital("Beta", dataset_Beta)
    hospital_Gamma = Hospital("Gamma", dataset_Gamma)
    """

    return hospitals

### Creation of hospitals

In [ ]:
import gc

# Your code here...

# Perform garbage collection
gc.collect()

66

In [14]:
set_reproducibility(seed=RANDOM_SEED)

In [15]:
hospitals = createHospitals()

### Checking shape

In [19]:
#{hospitals[name]["X_train"]}, {hospitals[name]["y_train"]}
for name in hospitals.keys():
  print(f"Hospitals {name} for X_test and y_test has a shape {hospitals[name]['X_test'].shape},{hospitals[name]['y_test'].shape}")
  print(f"Hospitals {name} for X_train and y_train has a shape {hospitals[name]['X_train'].shape},{hospitals[name]['y_train'].shape}")
  print()

Hospitals Alpha for X_test and y_test has a shape (895, 208, 176, 1),(895, 4)
Hospitals Alpha for X_train and y_train has a shape (2560, 208, 176, 1),(2560, 4)

Hospitals Beta for X_test and y_test has a shape (626, 208, 176, 1),(626, 4)
Hospitals Beta for X_train and y_train has a shape (1536, 208, 176, 1),(1536, 4)

Hospitals Gamma for X_test and y_test has a shape (438, 208, 176, 1),(438, 4)
Hospitals Gamma for X_train and y_train has a shape (1025, 208, 176, 1),(1025, 4)



## Creation of FedProx Model

## Testing function

In [20]:
def get_X_test():
    X_test = np.concatenate(
            [hospitals[hospital_name]["X_test"]
            for hospital_name in HOSPITAL_SPLIT],
            axis=0,)
    return X_test


def get_y_test():
    y_test = np.concatenate(
            [hospitals[hospital_name]["y_test"]
            for hospital_name in HOSPITAL_SPLIT],
            axis=0,)
    return y_test

### Fitting function

In [21]:
def fitting_model(_hospital_name, model, epochs):

    """fitting the model"""
    X_train = hospitals[_hospital_name]["X_train"]
    y_train = hospitals[_hospital_name]["y_train"]
    X_val = hospitals[_hospital_name]["X_val"]
    y_val = hospitals[_hospital_name]["y_val"]
    X_test = hospitals[_hospital_name]["X_test"]
    y_test = hospitals[_hospital_name]["y_test"]
    for epoch in range(epochs):
        # Training phase
        train_losses = []
        for i in range(0, len(X_train), BATCH_SIZE):
            batch_x = X_train[i:i + BATCH_SIZE]
            batch_y = y_train[i:i + BATCH_SIZE]

            train_loss = model.train_step(batch_x, batch_y)
            train_losses.append(train_loss)

        # Validation phase
        val_losses = []
        val_accuracies = []
        for i in range(0, len(X_val), BATCH_SIZE):
            val_batch_x = X_val[i:i + BATCH_SIZE]
            val_batch_y = y_val[i:i + BATCH_SIZE]

            loss, accuracy = model.evaluation_step(val_batch_x, val_batch_y)
            val_losses.append(loss)
            val_accuracies.append(accuracy)

        mean_train_loss = np.mean(train_losses)
        mean_val_loss = np.mean(val_losses)
        mean_accuracy_val = np.mean(val_accuracies)
        print(f"Epoch {epoch + 1}, Training Loss={mean_train_loss}, Validation Loss={mean_val_loss}, Val accuracy={mean_accuracy_val}")



## Aggregation

In [22]:
def similarity_single(
    _hospital_address, _hospitals_weights, _averaged_weights, _hospitals_addresses
):
    numerator = [
        np.linalg.norm(h_w - a_w)
        for hospital_address in _hospitals_addresses
        for h_w, a_w in zip(_hospitals_weights[hospital_address], _averaged_weights)
    ]
    numerator = sum(numerator)

    denominator = [
        np.linalg.norm(h_w - a_w)
        for h_w, a_w in zip(_hospitals_weights[_hospital_address], _averaged_weights)
    ]
    denominator = sum(denominator) + (10**-5)

    result = numerator / denominator
    return result

In [23]:
def similarity_factor_single(
    _hospital_address, _hospital_weights, _averaged_weights, _hospitals_addresses
):
    return similarity_single(
        _hospital_address, _hospital_weights, _averaged_weights, _hospitals_addresses
    ) / sum(
        [
            similarity_single(
                hospital_address,
                _hospital_weights,
                _averaged_weights,
                _hospitals_addresses,
            )
            for hospital_address in _hospitals_addresses
        ]
    )

In [24]:
def federated_learning(hospitals_weights):
    # computing the AVERAGE of the collaborators weights
    hospitals_addresses = list(models.keys())
    hospitals_number = len(hospitals_weights)
    weights_dim = len(hospitals_weights[list(hospitals_weights.keys())[0]])

    averaged_weights = []
    for i in range(weights_dim):
        layer_weights = []
        for hospital_address in hospitals_weights:
            layer_weights.append(hospitals_weights[hospital_address][i])
        averaged_weights.append(sum(layer_weights) / hospitals_number)

    for i in range(len(averaged_weights)):
        averaged_weights[i] = np.array(
            averaged_weights[i]
        )  # convert the list to a NumPy array

    #for hospital_address in hospitals_addresses:
    #    print_weights(hospitals_weights[hospital_address])
    #print_weights(averaged_weights)

    # computing the similarity factors
    factors = dict.fromkeys(hospitals_addresses, 0)
    for hospital_address in hospitals_addresses:
        if SIMILARITY == 'single':
            factors[hospital_address] = similarity_factor_single(
                hospital_address, hospitals_weights, averaged_weights, hospitals_addresses
            )
        #else:
        #    factors[hospital_address] = similarity_factor_multiple(
        #        hospital_address, hospitals_weights, averaged_weights, models
        #    )
    print("SIMILARITY FACTORS: ")
    for hospital_address in factors:
        print(
            f"Hospital address: {hospital_address}\tSimilarity factor: {factors[hospital_address]}"
        )

    # computing the AGGREGATION of the collaborators weights
    aggregated_weights = []

    for i in range(weights_dim):
        layer_weights = []
        for hospital_address in hospitals_addresses:
            if SIMILARITY == 'single':
                layer_weights.append(
                    factors[hospital_address] * hospitals_weights[hospital_address][i]
                )
            elif SIMILARITY == 'multiple':
                layer_weights.append(
                    factors[hospital_address][i] * hospitals_weights[hospital_address][i]
                )
        aggregated_weights.append(sum(layer_weights))

    for i in range(len(aggregated_weights)):
        aggregated_weights[i] = np.array(
            aggregated_weights[i]
        )  # Convert the list to a NumPy array

    # for TEST purposes:    compare AGGREGATED and AVERAGED parameters performance
    aggregated_weights = averaged_weights
    """
    # for SIMULATION/EVALUATION purposes
    """
    model_test.set_weights(averaged_weights)
    results = model_test.predict(X_test)
    y_predicted = list(map(np.argmax, results))
    labels_y_test = np.argmax(y_test, axis=1)
    FL_classification_report.append(
        classification_report(
            labels_y_test,
            y_predicted,  # labels=LABELS
        )
    )
    #print("y_predicted: ",  y_predicted)
    #print("y_test: ", labels_y_test)
    FL_evaluation.append(model_test.evaluate(X_test, y_test))
    return aggregated_weights

## Creation of 3 model

In [ ]:
NUM_EPOCHS=10
NUM_ROUNDS=10

In [ ]:
model_alpha.compile(optimizer="adam", metrics="accuracy")
model_beta.compile(optimizer="adam", metrics="accuracy")
model_gamma.compile(optimizer="adam", metrics="accuracy")

NameError: ignored

In [ ]:
models = {"Alpha":model_alpha, "Beta":model_beta, "Gamma":model_gamma}

NameError: ignored

## Testing

In [ ]:
FL_evaluation = []
FL_classification_report = []
X_test = get_X_test()
y_test = get_y_test()
model_test = FedAvg() # cambiare
model_test.build((None, WIDTH, HEIGHT, DEPTH))
model_test.compile(optimizer="adam", metrics="accuracy")
NUM_ROUNDS = 5
for round in range(NUM_ROUNDS):
      print(f"FL ROUND {round+1}...")
      # reset the weights related events
      hospital_weights =  {}
      for name in HOSPITAL_SPLIT.keys():
          print("Fitting model ",name)
          fitting_model(name, models[name],NUM_EPOCHS)
          hospital_weights[name] = models[name].trainable_weights
      # continue after reception
      aggregated_weights = federated_learning(hospital_weights)
      # changing aggregator
      for name in HOSPITAL_SPLIT.keys():
          #models[name].set_weights(aggregated_weights)
          models[name].set_weights(aggregated_weights)
      print("------------------------------------")

## Show result

In [ ]:
for round in range(NUM_ROUNDS):
        print(f"Round {round+1}:\t Loss: {FL_evaluation[round][0]} - Accuracy: {FL_evaluation[round][1]:.3f}")
        print(FL_classification_report[round] + "\n")

Round 1:	 Loss: 0.0 - Accuracy: 0.684
              precision    recall  f1-score   support

           0       0.69      0.90      0.78       385
           1       0.62      0.55      0.58       267
           2       1.00      0.31      0.47       111
           3       0.00      0.00      0.00         5

    accuracy                           0.68       768
   macro avg       0.58      0.44      0.46       768
weighted avg       0.71      0.68      0.66       768


Round 2:	 Loss: 0.0 - Accuracy: 0.854
              precision    recall  f1-score   support

           0       0.94      0.80      0.87       385
           1       0.77      0.91      0.84       267
           2       0.83      0.94      0.88       111
           3       0.00      0.00      0.00         5

    accuracy                           0.85       768
   macro avg       0.64      0.66      0.65       768
weighted avg       0.86      0.85      0.85       768


Round 3:	 Loss: 0.0 - Accuracy: 0.904
              

In [ ]:
len(model_test.trainable_weights)

10

## Paper Implementation FEDPROX

In [25]:
def process_grad(grads):
    '''
    Args:
        grads: grad
    Return:
        a flattened grad in numpy (1-D array)
    '''

    client_grads = grads[0]

    for i in range(1, len(grads)):
        client_grads = np.append(client_grads, grads[i]) # output a flattened array


    return client_grads

In [49]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.initializers import GlorotUniform, Zeros
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    Rescaling,
)
import numpy as np
import random


class FedAvg(tf.keras.Model):

    def __init__(self, num_classes=4, random_seed=42):
        super(FedAvg, self).__init__()
        set_reproducibility(random_seed)


        # Layers
        self.glorot_initializer = GlorotUniform(seed=random_seed)
        self.rescale_layer = Rescaling(1.0 / 255)
        self.conv1 = Conv2D(
            16,
            3,
            padding="same",
            activation="relu",
            kernel_initializer=self.glorot_initializer)
        self.pool = MaxPooling2D((2, 2))
        self.conv2 = Conv2D(32, 3, padding="same", activation="relu", kernel_initializer=self.glorot_initializer)
        self.conv3 = Conv2D(
            64,
            3,
            padding="same",
            activation="relu",
            kernel_initializer=self.glorot_initializer,
        )
        self.flatten_layer = Flatten()
        self.dense1 = Dense(
            128,
            activation="relu",
            kernel_initializer=self.glorot_initializer,
            kernel_constraint=MaxNorm(3),
        )
        self.dropout_layer = Dropout(0.3, seed=random_seed)
        self.dense2 = Dense(
            num_classes,
            activation="softmax",
            kernel_initializer=self.glorot_initializer,
        )

    def call(self, inputs):

        x = self.rescale_layer(inputs)
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.pool(x)

        x = self.flatten_layer(x)
        x = self.dense1(x)
        x = self.dropout_layer(x)
        x = self.dense2(x)

        return x

    @tf.function
    def train_step(self, inputs, labels):
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = tf.keras.losses.categorical_crossentropy(labels, predictions)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss

    def val_step(self, x_val, y_val):
        predictions = self(x_val, training=False)  # Predict using the model

        # Calculate loss (you can use a different metric if needed)
        loss = tf.keras.losses.categorical_crossentropy(y_val, predictions)

        return loss



In [50]:
class FedProx(FedAvg):

    SERVER_WEIGHTS = None
    def __init__(self, num_classes=4, server_model=None, mu=0.01, random_seed=42):

        super(FedProx, self).__init__(num_classes, random_seed)

        self.server_model = server_model
        self.mu = tf.constant(mu, dtype=tf.float32)

    def set_server_model(self, server_model):
        self.server_model = server_model


    @tf.function
    def train_step(self, inputs, labels):
        with tf.GradientTape() as tape:
            #print("entro prima delle predictions")
            predictions = self(inputs, training=True)
            #print("entro dopo delle predictions")
            #print("gli aggregator non sono None e hanno valore",len(self.aggregator_weights))
            loss = tf.keras.losses.categorical_crossentropy(labels, predictions) + (self.mu/2 * self.difference_models_norm_2())

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss

    def difference_models_norm_2(self):
        """
        Return the norm 2 difference between the two model parameters
        """

        t1 = tf.concat([tf.reshape(value, [-1]) for value in self.trainable_weights],axis=0)
        t2 = tf.concat([tf.reshape(value, [-1]) for value in FedProx.SERVER_WEIGHTS],axis=0)
        assert len(t2) != 0
        diff = tf.subtract(t1, t2)
        squared_norm = tf.square(tf.norm(diff))
        #print("nella norm2: ",len(self.trainable_weights))
        #tf.print("Prox termd is different from zero = ",squared_norm != 0)
        #tf.print("Squared norm = ",squared_norm)
        return squared_norm

In [51]:
def aggregate(weights):
  total_weight = 0.0
  wsolns = list(weights.values())

  base = [0]*len(wsolns[0])
  print(len(base))
  for name, soln in weights.items():
      w = len(hospitals[name]["y_train"]) # w is the number of local samples
      total_weight += w
      s = np.array(soln)
      base += w*s


  averaged_soln = base / total_weight
  #print("Len avg = ",len(averaged_soln))
  return averaged_soln

In [52]:
num_classes = 4
mu = 0.01
model_alpha = FedProx(num_classes=num_classes,mu = mu )
model_beta = FedProx(num_classes=num_classes, mu = mu)
model_gamma = FedProx(num_classes=num_classes, mu = mu)
model_alpha.compile(optimizer="adam", metrics="accuracy")
model_beta.compile(optimizer="adam", metrics="accuracy")
model_gamma.compile(optimizer="adam", metrics="accuracy")



In [53]:
global_model = FedAvg(num_classes=num_classes)
global_model.build((None, WIDTH, HEIGHT, DEPTH))
global_model.compile(optimizer="adam", metrics="accuracy")
global_model.summary()

Model: "fed_avg_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_11 (Rescaling)    multiple                  0         
                                                                 
 conv2d_33 (Conv2D)          multiple                  160       
                                                                 
 max_pooling2d_11 (MaxPooli  multiple                  0         
 ng2D)                                                           
                                                                 
 conv2d_34 (Conv2D)          multiple                  4640      
                                                                 
 conv2d_35 (Conv2D)          multiple                  18496     
                                                                 
 flatten_11 (Flatten)        multiple                  0         
                                                         

In [54]:
assert len(global_model.trainable_weights) != 0 # check if it is empty

In [55]:
models = {"Alpha":model_alpha, "Beta":model_beta, "Gamma":model_gamma}

In [56]:
NUM_ROUNDS = 10
NUM_EPOCHS = 10

In [57]:
FedProx.SERVER_WEIGHTS = global_model.trainable_weights

In [58]:
def fitting_model(_hospital_name, model, epochs):

    """fitting the model"""
    X_train = hospitals[_hospital_name]["X_train"]
    y_train = hospitals[_hospital_name]["y_train"]
    X_val = hospitals[_hospital_name]["X_val"]
    y_val = hospitals[_hospital_name]["y_val"]
    X_test = hospitals[_hospital_name]["X_test"]
    y_test = hospitals[_hospital_name]["y_test"]
    for epoch in range(epochs):
        # Training phase

        for i in range(0, len(X_train), BATCH_SIZE):
            batch_x = X_train[i:i + BATCH_SIZE]
            batch_y = y_train[i:i + BATCH_SIZE]

            train_loss = model.train_step(batch_x, batch_y)

        # Validation phase

        for i in range(0, len(X_val), BATCH_SIZE):
            val_batch_x = X_val[i:i + BATCH_SIZE]
            val_batch_y = y_val[i:i + BATCH_SIZE]

            val_loss = model.val_step(val_batch_x, val_batch_y)

        mean_val_loss = np.mean(val_loss.numpy())

        mean_train_loss = np.mean(train_loss.numpy())

        print(f"Epoch {epoch + 1}, Training Loss={mean_train_loss:.4f}, Validation Loss={mean_val_loss:.4f}")


In [59]:
FL_evaluation = []
FL_classification_report = []
X_test = get_X_test()
y_test = get_y_test()

global_param = global_model.trainable_weights
for i in range(NUM_ROUNDS):
    print(f"Round = {i+1} ...")
    """
    model_len = process_grad(global_model.trainable_weights).size # fatto
    global_grads = np.zeros(model_len)
    client_grads = np.zeros(model_len)
    num_samples = []
    local_grads = []
    print("Model len = ",model_len)
    samples = {}

    for name in HOSPITAL_SPLIT.keys():
        X_train = hospitals[name]["X_train"]
        y_train = hospitals[name]["y_train"]
        num = len(y_train)
        samples[name] = num
        # Computing gradients
        batch_x = X_train[0:BATCH_SIZE]
        batch_y = y_train[0:BATCH_SIZE]
        client_grad = process_grad(models[name].get_grads(batch_x, batch_y))
        local_grads.append(client_grad)
        num_samples.append(num)
        global_grads = np.add(global_grads, client_grad * num)
    print("Grad = ",global_grads.shape)
    global_grads = global_grads * 1.0 / np.sum(np.asarray(num_samples))
    difference = 0
    for idx in range(len(HOSPITAL_SPLIT)):
        difference += np.sum(np.square(global_grads - local_grads[idx]))
        difference = difference * 1.0 / len(HOSPITAL_SPLIT)
    tqdm.write('gradient difference: {}'.format(difference))
    """
    csolns = []

    hospital_weights =  {} # reset_weights
    for name in HOSPITAL_SPLIT.keys():
        initial_weights = global_param
        #models[name].set_server_model(initial_weights)
        #total_iters = int(NUM_EPOCHS * samples[name] / BATCH_SIZE) + 2 # randint(low,high)=[low,high)

        # solve minimization locally

        epochs = random.randint(1, NUM_EPOCHS)
        print(f"Fitting model  {name} on {epochs} epochs ")

        if models[name].server_model != None:
          print("PESI DIVERSO DA NONE")

        fitting_model(name, models[name],epochs)
        hospital_weights[name] = models[name].trainable_weights
        print("numero di pesi salvati = ",len(models[name].trainable_weights))



    global_param = aggregate(hospital_weights)
    #global_param = federated_learning(hospital_weights)
    #assert(len(global_param) == len(models["Alpha"].trainable_weights))
    global_model.set_weights(global_param)
    assert global_param != initial_weights
    FedProx.SERVER_WEIGHTS = global_param
    #self.client_model.set_params(param_global)
    # TESTING

    results = global_model.predict(X_test)
    y_predicted = list(map(np.argmax, results))
    labels_y_test = np.argmax(y_test, axis=1)
    FL_classification_report.append(
        classification_report(
            labels_y_test,
            y_predicted,  # labels=LABELS
        )
    )
    FL_evaluation.append(global_model.evaluate(X_test, y_test))


Round = 1 ...
Fitting model  Alpha on 2 epochs 
Epoch 1, Training Loss=0.9737, Validation Loss=0.9743
Epoch 2, Training Loss=0.8592, Validation Loss=0.9986
numero di pesi salvati =  10
Fitting model  Beta on 1 epochs 
Epoch 1, Training Loss=1.1893, Validation Loss=1.1797
numero di pesi salvati =  10
Fitting model  Gamma on 5 epochs 
Epoch 1, Training Loss=0.9015, Validation Loss=1.1351
Epoch 2, Training Loss=0.4440, Validation Loss=1.1347
Epoch 3, Training Loss=0.4373, Validation Loss=1.1622
Epoch 4, Training Loss=0.5697, Validation Loss=1.0617
Epoch 5, Training Loss=0.6334, Validation Loss=1.3206
numero di pesi salvati =  10
10


<ipython-input-51-22a2dd575a43>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  s = np.array(soln)
<ipython-input-59-6458138154fc>:64: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  assert global_param != initial_weights
<ipython-input-59-6458138154fc>:64: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  assert global_param != initial_weights


62/62 [==============================] - 1s 7ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


62/62 [==============================] - 1s 8ms/step - loss: 0.0000e+00 - accuracy: 0.4926
Round = 2 ...
Fitting model  Alpha on 4 epochs 
Epoch 1, Training Loss=0.7831, Validation Loss=0.9660
Epoch 2, Training Loss=0.8139, Validation Loss=1.0451
Epoch 3, Training Loss=0.7756, Validation Loss=1.0461
Epoch 4, Training Loss=0.7789, Validation Loss=0.9911
numero di pesi salvati =  10
Fitting model  Beta on 4 epochs 
Epoch 1, Training Loss=1.0125, Validation Loss=1.1311
Epoch 2, Training Loss=1.0819, Validation Loss=1.0242
Epoch 3, Training Loss=0.9597, Validation Loss=0.9498
Epoch 4, Training Loss=0.9149, Validation Loss=0.9385
numero di pesi salvati =  10
Fitting model  Gamma on 3 epochs 
Epoch 1, Training Loss=0.3600, Validation Loss=1.0710
Epoch 2, Training Loss=0.5844, Validation Loss=1.1535
Epoch 3, Training Loss=0.6050, Validation Loss=1.0095
numero di pesi salvati =  10
10


<ipython-input-51-22a2dd575a43>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  s = np.array(soln)
<ipython-input-59-6458138154fc>:64: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  assert global_param != initial_weights


62/62 [==============================] - 1s 7ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


62/62 [==============================] - 1s 8ms/step - loss: 0.0000e+00 - accuracy: 0.6039
Round = 3 ...
Fitting model  Alpha on 2 epochs 
Epoch 1, Training Loss=0.6350, Validation Loss=1.0296
Epoch 2, Training Loss=0.5807, Validation Loss=0.9541
numero di pesi salvati =  10
Fitting model  Beta on 9 epochs 
Epoch 1, Training Loss=0.8407, Validation Loss=0.8841


KeyboardInterrupt: ignored

# Mu 0.01

In [ ]:
for round in range(NUM_ROUNDS):
    print(f"Round {round+1}:\t Loss: {FL_evaluation[round][0]} - Accuracy: {FL_evaluation[round][1]:.3f}")
    print(FL_classification_report[round] + "\n")

Round 1:	 Loss: 0.0 - Accuracy: 0.501
              precision    recall  f1-score   support

           0       0.50      1.00      0.67       385
           1       0.50      0.00      0.01       267
           2       0.00      0.00      0.00       111
           3       0.00      0.00      0.00         5

    accuracy                           0.50       768
   macro avg       0.25      0.25      0.17       768
weighted avg       0.43      0.50      0.34       768


Round 2:	 Loss: 0.0 - Accuracy: 0.577
              precision    recall  f1-score   support

           0       0.62      0.84      0.71       385
           1       0.47      0.40      0.43       267
           2       0.92      0.10      0.18       111
           3       0.00      0.00      0.00         5

    accuracy                           0.58       768
   macro avg       0.50      0.34      0.33       768
weighted avg       0.60      0.58      0.53       768


Round 3:	 Loss: 0.0 - Accuracy: 0.665
              

# Mu 0.1

In [ ]:
for round in range(NUM_ROUNDS):
    print(f"Round {round+1}:\t Loss: {FL_evaluation[round][0]} - Accuracy: {FL_evaluation[round][1]:.3f}")
    print(FL_classification_report[round] + "\n")

Round 1:	 Loss: 0.0 - Accuracy: 0.501
              precision    recall  f1-score   support

           0       0.50      1.00      0.67       385
           1       0.00      0.00      0.00       267
           2       0.00      0.00      0.00       111
           3       0.00      0.00      0.00         5

    accuracy                           0.50       768
   macro avg       0.13      0.25      0.17       768
weighted avg       0.25      0.50      0.33       768


Round 2:	 Loss: 0.0 - Accuracy: 0.501
              precision    recall  f1-score   support

           0       0.53      0.94      0.67       385
           1       0.29      0.09      0.14       267
           2       0.00      0.00      0.00       111
           3       0.00      0.00      0.00         5

    accuracy                           0.50       768
   macro avg       0.20      0.26      0.20       768
weighted avg       0.36      0.50      0.39       768


Round 3:	 Loss: 0.0 - Accuracy: 0.522
              

## Mu 0.001

In [ ]:
for round in range(NUM_ROUNDS):
    print(f"Round {round+1}:\t Loss: {FL_evaluation[round][0]} - Accuracy: {FL_evaluation[round][1]:.3f}")
    print(FL_classification_report[round] + "\n")

Round 1:	 Loss: 0.0 - Accuracy: 0.540
              precision    recall  f1-score   support

           0       0.59      0.83      0.69       385
           1       0.43      0.36      0.39       267
           2       0.00      0.00      0.00       111
           3       0.00      0.00      0.00         5

    accuracy                           0.54       768
   macro avg       0.25      0.30      0.27       768
weighted avg       0.44      0.54      0.48       768


Round 2:	 Loss: 0.0 - Accuracy: 0.638
              precision    recall  f1-score   support

           0       0.73      0.77      0.75       385
           1       0.51      0.65      0.57       267
           2       0.95      0.18      0.30       111
           3       0.00      0.00      0.00         5

    accuracy                           0.64       768
   macro avg       0.55      0.40      0.41       768
weighted avg       0.68      0.64      0.62       768


Round 3:	 Loss: 0.0 - Accuracy: 0.724
              

# Mu 0.05

In [ ]:
for round in range(NUM_ROUNDS):
    print(f"Round {round+1}:\t Loss: {FL_evaluation[round][0]} - Accuracy: {FL_evaluation[round][1]:.3f}")
    print(FL_classification_report[round] + "\n")

Round 1:	 Loss: 0.0 - Accuracy: 0.503
              precision    recall  f1-score   support

           0       0.50      0.99      0.67       385
           1       0.50      0.01      0.02       267
           2       0.00      0.00      0.00       111
           3       0.00      0.00      0.00         5

    accuracy                           0.50       768
   macro avg       0.25      0.25      0.17       768
weighted avg       0.43      0.50      0.34       768


Round 2:	 Loss: 0.0 - Accuracy: 0.533
              precision    recall  f1-score   support

           0       0.59      0.79      0.68       385
           1       0.41      0.39      0.40       267
           2       0.00      0.00      0.00       111
           3       0.00      0.00      0.00         5

    accuracy                           0.53       768
   macro avg       0.25      0.30      0.27       768
weighted avg       0.44      0.53      0.48       768


Round 3:	 Loss: 0.0 - Accuracy: 0.548
              